In [1]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import pathlib
import torchvision.transforms as transforms
import torch
import PIL
from sklearn.model_selection import train_test_split
from torchvision.models import resnet
import sys
import pandas as pd
# from neptune import Context
from sklearn.metrics import f1_score
from neptune import Context
import torch.nn as nn
import torchvision.transforms as transforms
from torch import optim, save
from torch.utils.data import DataLoader
from torchvision.models import resnet50

from ignite.engine import Events
from ignite.engine import create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import CategoricalAccuracy, Recall, Precision
from ignite.metrics import Loss
import numpy as np
RANDOM_SEED = 666

LABEL_MAP = {
0: "Nucleoplasm" ,
1: "Nuclear membrane"   ,
2: "Nucleoli"   ,
3: "Nucleoli fibrillar center",   
4: "Nuclear speckles"   ,
5: "Nuclear bodies"   ,
6: "Endoplasmic reticulum"   ,
7: "Golgi apparatus"  ,
8: "Peroxisomes"   ,
9:  "Endosomes"   ,
10: "Lysosomes"   ,
11: "Intermediate filaments"  , 
12: "Actin filaments"   ,
13: "Focal adhesion sites"  ,
14: "Microtubules"   ,
15: "Microtubule ends"   ,
16: "Cytokinetic bridge"   ,
17: "Mitotic spindle"  ,
18: "Microtubule organizing center",  
19: "Centrosome",
20: "Lipid droplets"   ,
21: "Plasma membrane"  ,
22: "Cell junctions"   ,
23: "Mitochondria"   ,
24: "Aggresome"   ,
25: "Cytosol" ,
26: "Cytoplasmic bodies",
27: "Rods & rings"}


ctx = Context()
class MultiBandMultiLabelDataset(Dataset):
    BANDS_NAMES = ['_red.png','_green.png','_blue.png','_yellow.png']
    
    def __len__(self):
        return len(self.images_df)
    
    def __init__(self, images_df, 
                 base_path, 
                 image_transform,
                 augmentator=None,
                 train_mode=True,
                 n_channels=4
                ):
        if not isinstance(base_path, pathlib.Path):
            base_path = pathlib.Path(base_path)
            
        self.images_df = images_df.copy()
        self.image_transform = image_transform
        self.augmentator = augmentator
        self.images_df.Id = self.images_df.Id.apply(lambda x: base_path / x)
        self.mlb = MultiLabelBinarizer(classes=list(LABEL_MAP.keys()))
        self.train_mode = train_mode
        self.n_channels = n_channels

                                      
        
    def __getitem__(self, index):
        y = None
        X = self._load_multiband_image(index)
        if self.train_mode:
            y = self._load_multilabel_target(index)
        
        # augmentator can be for instance imgaug augmentation object
        if self.augmentator is not None:
            X = self.augmentator(X)
            
        X = self.image_transform(X)
            
        return X, y 
        
    def _load_multiband_image(self, index):
        row = self.images_df.iloc[index]
        image_bands = []
        for band_name in self.BANDS_NAMES:
            p = str(row.Id.absolute()) + band_name
            pil_channel = PIL.Image.open(p)
            image_bands.append(pil_channel)
            
        # lets pretend its a RBGA image to support 4 channels
        band4image = PIL.Image.merge('RGBA', bands=image_bands[:])
        return band4image
    
    
    def _load_multilabel_target(self, index):
        return list(map(int, self.images_df.iloc[index].Target.split(' ')))
    
        
    def collate_func(self, batch):
        labels = None
        images = [x[0] for x in batch]
        
        if self.train_mode:
            labels = [x[1] for x in batch]
            labels_one_hot  = self.mlb.fit_transform(labels)
            labels = torch.FloatTensor(labels_one_hot)
            
        image_stack = torch.stack(images)
        image_stack[:,0,:,:] + image_stack[:,3,:,:] / 2
        image_stack[:,1,:,:] + image_stack[:,3,:,:] / 2
        image_stack = image_stack[:,:3,:,:]

        
        
        return image_stack, labels

def get_model(n_classes, image_channels=4):
    model = resnet50(pretrained=False)
    for p in model.parameters():
        p.requires_grad = True
    inft = model.fc.in_features
    model.fc = nn.Linear(in_features=inft, out_features=n_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
    
    return model  


def train(trainer, train_loader, test_loader, epochs):
    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        iter = (engine.state.iteration - 1) % len(train_loader) + 1
#         ctx.channel_send('loss', engine.state.output)
        if iter % 100 == 0:
            print("Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
                  "".format(engine.state.epoch, iter, len(train_loader), engine.state.output))

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        evaluator.run(test_loader)
        metrics = evaluator.state.metrics
        avg_nll = metrics['loss']
        print("Training Results - Epoch: {}  Avg loss: {:.2f}"
              .format(engine.state.epoch, avg_nll))
        save(model, '/home/i008/model_{}_{}.torch'.format(engine.state.epoch, avg_nll))
    trainer.run(train_loader, max_epochs=epochs)
    
    return model 
    

# Eval
def evaluate(model, test_loader, threshold=0.2):
    all_preds = []
    true = []
    model.eval()
    for b in test_loader:
        X, y = b
        X, y = X.cuda(), y.cuda()
        pred = model(X)
        all_preds.append(pred.sigmoid().cpu().data.numpy())
        true.append(y.cpu().data.numpy())
        
        
    P = np.concatenate(all_preds)
    R = np.concatenate(true)
    
    for t in [0.05, 0.1, 0.15, 0.2, 0.25]:
        f1 = f1_score(P>t, R, average='macro')
        print(f1)
    return f1
    

## Submission
def predict_submission(model, submission_loader):
    all_preds = []
    model.eval()
    for i, b in enumerate(submission_loader):
        if i % 100: print('processing batch {}/{}'.format(i, len(submission_loader)))
        X, _ = b
        X = X.cuda()
        pred = model(X)
        all_preds.append(pred.sigmoid().cpu().data.numpy())
    return np.concatenate(all_preds)
        
         
def make_submission_file(sample_submission_df, predictions):
    submissions = []
    for row in predictions:
        subrow = ' '.join(list([str(i) for i in np.nonzero(row)[0]]))
        submissions.append(subrow)
    
    sample_submission_df['Predicted'] = submissions
    sample_submission_df.to_csv('submission.csv', index=None)
    
    
    return sample_submission_df

                                                                
                                                                    
def get_resnet(n_classes=28, resnet_version='resnet50', image_channels=3, pretrained=True):
    """
    :param n_classes:
    :param resnet_version in ['resnet152', 'resnet50', 'resnet18', 'resnet101']
    :param image_channels:
    :param pretrained:
    :return:
    """
    assert resnet_version in ['resnet' + str(i) for i in [18, 50, 101, 152]]
    model = getattr(resnet, resnet_version)(pretrained=pretrained)
    for p in model.parameters():
        p.requires_grad = True
    inft = model.fc.in_features
    model.fc = nn.Linear(in_features=inft, out_features=n_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    if image_channels is not 3:
        model.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)

    return model                                                                    
    
PATH_TO_IMAGES = '/media/i008/duzy/genom/train/'
PATH_TO_TEST_IMAGES = '/media/i008/duzy/genom/test/'
PATH_TO_META = '/media/i008/duzy/genom/train.csv'
SAMPLE_SUBMI = '/media/i008/duzy/genom/sample_submission.csv'


neptune: Executing in Offline Mode.


In [2]:
SEED = 666
DEV_MODE = False
    
df = pd.read_csv(PATH_TO_META)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=SEED)
df_submission = pd.read_csv(SAMPLE_SUBMI)

if DEV_MODE:
    df_train, df_test = df_train[:1000], df_test[:100]
    

SIZE = 256
BS= 16

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

image_transform_train = transforms.Compose([
           transforms.Resize(SIZE),
           transforms.RandomVerticalFlip(),
           transforms.RandomHorizontalFlip(),
           transforms.RandomRotation(90),
           transforms.ToTensor(),
           transforms.Normalize(mean=MEAN, std=STD)


       ])


image_transform_test = transforms.Compose([
           transforms.Resize(SIZE),
#             transforms.RandomVerticalFlip(),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomRotation(90)
           transforms.ToTensor(),
           transforms.Normalize(mean=MEAN, std=STD)


       ])
gtrain = MultiBandMultiLabelDataset(df_train, base_path=PATH_TO_IMAGES, image_transform=image_transform_train)
gtest = MultiBandMultiLabelDataset(df_test, base_path=PATH_TO_IMAGES, image_transform=image_transform_test)
gsub = MultiBandMultiLabelDataset(df_submission, base_path=PATH_TO_TEST_IMAGES, train_mode=False, image_transform=image_transform_test)

train_load = DataLoader(gtrain, collate_fn=gtrain.collate_func, batch_size=BS, num_workers=6)
test_load = DataLoader(gtest, collate_fn=gtest.collate_func, batch_size=BS, num_workers=6)
submission_load = DataLoader(gsub, collate_fn=gsub.collate_func, batch_size=BS, num_workers=6)

model = get_resnet()
device='cuda'
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.cuda()
# evaluator = create_supervised_evaluator(model,
#                                             device=device,
#                                             metrics={'loss': Loss(criterion)
#                                                     })
optimizer = optim.Adam(filter(lambda p: p.requires_grad,model.parameters()), lr=0.00005)
# trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

In [3]:
# trainer.add_event_handler()

# TRAIN

In [4]:
# train the model
# train(trainer, train_load, test_load, epochs=10)

# EVAL

In [5]:
# model = torch.load('/home/i008/model_1_0.1601224570123783.torch')

# res = evaluate(model, test_load, threshold=0.2)

# SUBMISSION

In [6]:
# submission_predictions=predict_submission(model, submission_load)

In [7]:
# prepare the submission file and 
# THRESHOLD = 0.05
# p = submission_predictions>THRESHOLD

# submission_file = make_submission_file(sample_submission_df=df_submission,
#                      predictions=p)

In [8]:
from ignite.engine import create_supervised_trainer
from torchvision.models import resnet
from torch import nn
from ignite.engine import Events
import os 

def get_resnet(n_classes=27, resnet_version='resnet152', image_channels=4, pretrained=True):
    """
    :param n_classes:
    :param resnet_version in ['resnet152', 'resnet50', 'resnet18', 'resnet101']
    :param image_channels:
    :param pretrained:
    :return:
    """
    assert resnet_version in ['resnet' + str(i) for i in [18, 50, 101, 152]]
    model = getattr(resnet, resnet_version)(pretrained=pretrained)
    for p in model.parameters():
        p.requires_grad = True
    inft = model.fc.in_features
    model.fc = nn.Linear(in_features=inft, out_features=n_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    if image_channels is not 3:
        model.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)

    return model
from neptune import Context

neptune_context = Context()

class ModelTrainer:
    def __init__(self, model,
                 optimizer,
                 criterion,
                 train_loader,
                 test_loader,
                 device,
                 epochs,
                 checkpoint_directory='/home/i008/'
        ):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.trainer = create_supervised_trainer(model, optimizer, criterion, device)
        self.epochs = epochs
        self.checkpoint_directory = checkpoint_directory
        self.epoch_end_loss = None
        self.evaluator = create_supervised_evaluator(model, device=device, 
                                                metrics={'loss': Loss(criterion) })
        
        self.register_callbacks()


    def train(self):
        self.trainer.run(self.train_loader, max_epochs=self.epochs)

    def register_callbacks(self):
        self.trainer.add_event_handler(Events.ITERATION_COMPLETED, self._callback_store_training_loss)
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self._callback_store_training_results)
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self._callback_checkpoint)
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self._epoch_end_callback_log)
        self.trainer.add_event_handler(Events.ITERATION_COMPLETED, self._batch_end_callback_log)
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self._batch_end_evaluate)
        

    def _callback_store_training_loss(self, engine):
        self.current_epoch = engine.state.epoch
        self.batch_end_loss = engine.state.output
        self.batch_number = engine.state.iteration
        
         
    def _callback_store_training_results(self, engine):
        if self.test_loader is not None:
            self.evaluator.run(self.test_loader)
            metrics = self.evaluator.state.metrics
            avg_nll = metrics['loss']
            self.epoch_end_loss = avg_nll
        
    
    def _callback_checkpoint(self, *args):
        model_name = str(model).split('(')[0]
        model_name += '_{}_{}'
        model_path = os.path.join(self.checkpoint_directory, model_name).format(self.current_epoch, self.epoch_end_loss)
        print("Storing model {}".format(model_path))
        save(self.model, model_path)
        
        
    def _epoch_end_callback_log(self, *args):
        neptune_context.channel_send('validation_loss_epoch_end', self.epoch_end_loss)
        neptune_context.channel_send("epoch", self.current_epoch)
        print("Finished epoch {} with loss {}".format(self.current_epoch, self.epoch_end_loss))
            
    def _batch_end_callback_log(self, *args):
        neptune_context.channel_send('loss', self.batch_end_loss) 
        if self.batch_number % 100 == 0:
            print(self.batch_end_loss, self.batch_number)
        
    def _batch_end_evaluate(self, *args):
        def evaluate_f1(model, test_loader, threshold=0.2):
            all_preds = []
            true = []
            model.eval()
            for b in test_loader:
                X, y = b
                X, y = X.cuda(), y.cuda()
                pred = model(X)
                all_preds.append(pred.sigmoid().cpu().data.numpy())
                true.append(y.cpu().data.numpy())


            P = np.concatenate(all_preds)
            R = np.concatenate(true)
            
            f_scores = []
            for t  in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]:
                f1 = f1_score(P>t, R, average='macro')
                f_scores.append(f1)
                
            return f_scores
                
        self.f1_score = evaluate_f1(self.model, self.test_loader)
        self.model.train()
        
        
        print("Evaluation score is {}".format(self.f1_score))

        
        

neptune: Executing in Offline Mode.


In [9]:
mlc = ModelTrainer(model, optimizer, criterion, train_load, test_load, device, 10)

In [10]:
mlc.train()

0.14000296592712402 100
0.1540842354297638 200
0.1747194081544876 300
0.15312162041664124 400
0.17220529913902283 500


Process Process-2:
Process Process-1:
Process Process-4:
Process Process-3:
Process Process-5:
Traceback (most recent call last):
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/multiprocessing/process.py", line 93, in run
  

Traceback (most recent call last):
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-26cc0647907d>", line 1, in <module>
    mlc.train()
  File "<ipython-input-8-9aad6f51e451>", line 58, in train
    self.trainer.run(self.train_loader, max_epochs=self.epochs)
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/site-packages/ignite/engine/engine.py", line 223, in run
    self._handle_exception(e)
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/site-packages/ignite/engine/engine.py", line 188, in _handle_exception
    raise e
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/site-packages/ignite/engine/engine.py", line 210, in run
    hours, mins, secs = self._run_once_on_dataset()
  File "/home/i008/anaconda3/envs/genom/lib/python3.6/site-packages/ignite/engine/engine.py", line 177, in _run_once_on_dataset
    self._handl

KeyboardInterrupt: 

In [ ]:
800 * 16
